In [21]:
import pandas as pd
from utilities.functions import *
from utilities.luts import *

In [22]:
# CSV with the list of anchorage neighborhoods to get data for
anc = pd.read_csv(
    "/Users/joshpaul/epa-justice/repo/epa-justice/tbl/Anchorage_CensusTracts_2020.csv"
)
anc.head()

,GEOIDFQ,TractNo,TractName
0,1400000US02020000500,Census Tract 5,Government Hill
1,1400000US02020000601,Census Tract 6.01,West Mountain View/Ship Creek
2,1400000US02020000602,Census Tract 6.02,East Mountain View
3,1400000US02020000701,Census Tract 7.01,Ptarmigan Area
4,1400000US02020000702,Census Tract 7.02,Northwest Muldoon


In [23]:
# to match the standard input table, we want these columns in the dataframe:
# id,name,alt_name,region,country,latitude,longitude,type,GEOIDFQ,PLACENAME,AREATYPE,COMMENT
# map GEOIDFQ to GEOIDFQ, TractNo to PLACENAME, and TrachName to name

anc = anc.rename(
    columns={"GEOIDFQ": "GEOIDFQ", "TractNo": "PLACENAME", "TractName": "name"}
)

# add the region, country, latitude, longitude, type, AREATYPE, COMMENT columns
anc["id"] = None
anc["alt_name"] = None
anc["region"] = "Alaska"
anc["country"] = "US"
anc["latitude"] = None
anc["longitude"] = None
anc["type"] = "neighborhood"
anc["AREATYPE"] = "Census tract"
anc["COMMENT"] = None

# make up some sequential IDs... we need these to for the data fetching functions
# ID's should start with "ANC"
anc["id"] = ["ANC" + str(i).zfill(3) for i in range(1, len(anc) + 1)]

In [24]:
anc.head()

,GEOIDFQ,PLACENAME,name,id,alt_name,region,country,latitude,longitude,type,AREATYPE,COMMENT
0,1400000US02020000500,Census Tract 5,Government Hill,ANC001,None,Alaska,US,None,None,neighborhood,Census tract,None
1,1400000US02020000601,Census Tract 6.01,West Mountain View/Ship Creek,ANC002,None,Alaska,US,None,None,neighborhood,Census tract,None
2,1400000US02020000602,Census Tract 6.02,East Mountain View,ANC003,None,Alaska,US,None,None,neighborhood,Census tract,None
3,1400000US02020000701,Census Tract 7.01,Ptarmigan Area,ANC004,None,Alaska,US,None,None,neighborhood,Census tract,None
4,1400000US02020000702,Census Tract 7.02,Northwest Muldoon,ANC005,None,Alaska,US,None,None,neighborhood,Census tract,None


In [25]:
# run the fetch and merge function to get the data for these neighborhoods
# we don't need to aggregate rows since there are no duplicate IDs here
anc_results_df = run_fetch_and_merge(anc)
anc_results_df

In [ ]:
# TODO: reformat the dataframe to match the CSV output from the API
# we need to import some of the lookup tables from the API in order to do this

In [ ]:
# save to CSV
anc_results_df.to_csv("tbl/anc_neighborhood_demographics.csv", index=False)